In [1]:
!pip install bs4

In [10]:
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup

In [17]:
all_news = []
for i in range(1, 5):
  r = requests.get(f'https://www.pagina12.com.ar/secciones/economia?page={i}')
  soup = BeautifulSoup(r.text, 'html.parser')
  articles = soup.find_all('article', class_='article-item article-item--teaser ')
  for article in articles:
    anchor = article.find('a', class_='p12-separator--left--primary')
    if anchor is None:
      print(article)
      continue
    r_article = requests.get(f'https://www.pagina12.com.ar/{anchor["href"]}')
    soup_article = BeautifulSoup(r_article.text, 'html.parser')
    paragraphs = soup_article.find('div', class_='article-main-content article-text ').find_all('p')
    news =''
    for paragraph in paragraphs:
      if paragraph.string is not None:
        news += paragraph.string
    all_news.append(news)
    time.sleep(0)

In [18]:
all_news

[' “Teníamos un endeudamiento insostenible en moneda extranjera. La carga de la deuda estrangulaba toda posibilidad de crecimiento económico y desarrollo“, dijo el titular del Palacio de Hacienda ante la Bicameral de Seguimiento de la Deuda Externa en el Congreso.  En su presentación planteó que “la deuda en moneda extranjera es un ataque a la capacidad de implementar políticas públicas”. Al mismo tiempo mencionó que “cuando se toma una deuda con el Fondo, el problema es mayor”. El ministro se mostró crítico con la oposición por las falacias que hace circular sobre temas vinculados con el endeudamiento. “Hay un expresidente y una exgobernadora que han marcado que el endeudamiento es un problema de nuestra administración, y que nos estamos endeudando más que la administración precedente. Sus números son incorrectos”. Guzmán explicó que el Gobierno actual no se está endeudando en dólares. “Lo que ha hecho nuestro Gobierno es financiar déficits fiscales de la magnitud que eran necesarios 